In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style("whitegrid")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 200)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Tennis-Data

In [2]:
# Set the directory path where the XLS and XLSX files are located
folder_path = "_data"

# Get a list of all XLS and XLSX files in the directory
file_list = [file for file in os.listdir(folder_path) if file.startswith("tennis_data_20") and (file.endswith(".xls") or file.endswith(".xlsx"))]

# Initialize an empty list to store the dataframes
dataframes = []
single_df = []
df_uk = []

# Iterate over each XLS and XLSX file
for file in file_list:
    file_path = os.path.join(folder_path, file)
    year_pos = file.find("20")
    year = file[year_pos:year_pos+4]
    single_df = pd.read_excel(file_path)
    date_index = single_df.columns.get_loc('Date')
    single_df.insert(date_index + 1, 'Year', int(year))
    dataframes.append(single_df)

# Merge all dataframes into a single dataframe
df_uk = pd.concat(dataframes, ignore_index=True)

# Print the merged dataframe
print(df_uk.head())

   ATP  Location        Tournament          Date     Year  Series  Court   Surface   Round     Best of     Winner        Loser      WRank LRank    WPts    LPts   W1   L1   W2   L2   W3   L3   W4  \
0   1   Brisbane  Brisbane International 2012-12-31  2013  ATP250  Outdoor   Hard   1st Round   3.00        Mayer F.    Giraldo S. 28.00   57.00 1215.00  778.00 6.00 4.00 6.00 4.00  NaN  NaN NaN   
1   1   Brisbane  Brisbane International 2012-12-31  2013  ATP250  Outdoor   Hard   1st Round   3.00     Nieminen J.  Benneteau J. 41.00   35.00  927.00 1075.00 6.00 3.00 2.00 6.00 6.00 1.00 NaN   
2   1   Brisbane  Brisbane International 2012-12-31  2013  ATP250  Outdoor   Hard   1st Round   3.00    Nishikori K.  Matosevic M. 19.00   49.00 1830.00  845.00 7.00 5.00 6.00 2.00  NaN  NaN NaN   
3   1   Brisbane  Brisbane International 2012-12-31  2013  ATP250  Outdoor   Hard   1st Round   3.00    Baghdatis M.   Mitchell B. 36.00  326.00 1070.00  137.00 6.00 4.00 6.00 4.00  NaN  NaN NaN   
4   1   Br

In [3]:
print(df_uk.shape)

# trim Location and Tournament fields to remove leading and trailing spaces
df_uk['Location'] = df_uk['Location'].str.strip()
df_uk['Tournament'] = df_uk['Tournament'].str.strip()
df_uk['Winner'] = df_uk['Winner'].str.strip()
df_uk['Loser'] = df_uk['Loser'].str.strip()

# remove data points with Date <= 2008 and Date > 2022
df_uk['Date'] = pd.to_datetime(df_uk['Date'], format='%Y%m%d')
df_uk['Date'] = df_uk['Date'].dt.strftime('%Y-%m-%d')
df_uk = df_uk[(df_uk['Year'] > 2008) & (df_uk['Year'] <= 2022)]
df_uk = df_uk.sort_values(by=['Date', 'ATP'])
print(df_uk.shape)

# insert a new column 'tourney_name' based on the 'Location' and 'Tournament' columns
df_tour = pd.read_excel(folder_path + '/_tournaments_list.xlsx')
tournaments = df_tour.set_index(["df_uk['Location']", "df_uk['Tournament']"])["df_uk['tourney_name']"].to_dict()
def get_tourney_name(row):
    tour = tournaments.get((row['Location'], row['Tournament']))
    return tour if tour else 'NA'
tourney_name = df_uk.apply(get_tourney_name, axis=1)
df_uk.insert(3, 'tourney_name', tourney_name)

# insert a new column 'round' based on the 'Round' column
def map_rounds(group):
    if '4th Round' in group['Round'].values:
        round_dict = {'1st Round': 'R128', '2nd Round': 'R64', '3rd Round': 'R32', '4th Round': 'R16'}
    elif '3rd Round' in group['Round'].values:
        round_dict = {'1st Round': 'R64', '2nd Round': 'R32', '3rd Round': 'R16'}
    elif '2nd Round' in group['Round'].values:
        round_dict = {'1st Round': 'R32', '2nd Round': 'R16'}
    else:
        round_dict = {}
    round_dict.update({'Round Robin': 'RR', 'Quarterfinals': 'QF', 'Semifinals': 'SF', 'The Final': 'F'})
    group['round'] = group['Round'].map(round_dict)
    return group
df_uk = df_uk.groupby(['tourney_name', 'Year']).apply(map_rounds)
cols = df_uk.columns.tolist()
index = cols.index('Round') + 1
cols = cols[:index] + ['round'] + cols[index:-1]
df_uk = df_uk.reindex(columns=cols)

# correct Surface error from 'Carpet' to 'Hard' for tourney_name 'Zagreb' in year 2009
df_uk.loc[(df_uk['tourney_name'] == 'Zagreb') & (df_uk['Year'] == 2009), 'Surface'] = 'Hard'

# correct tourney_name exchange error between Auckland and Adelaide for year 2020
df_uk.loc[(df_uk['tourney_name'] == 'Auckland') & (df_uk['Year'] == 2020), 'Location'] = 'Adelaide'
df_uk.loc[(df_uk['tourney_name'] == 'Auckland') & (df_uk['Year'] == 2020), 'Tournament'] = 'Adelaide International'
df_uk.loc[(df_uk['tourney_name'] == 'Auckland') & (df_uk['Year'] == 2020), 'tourney_name'] = 'Adelaide'
df_uk.loc[(df_uk['ATP'] == 3) & (df_uk['Year'] == 2020), 'Location'] = 'Auckland' # ATP == 3 means 'Adelaide' tournament
df_uk.loc[(df_uk['ATP'] == 3) & (df_uk['Year'] == 2020), 'Tournament'] = 'ASB Classic'
df_uk.loc[(df_uk['ATP'] == 3) & (df_uk['Year'] == 2020), 'tourney_name'] = 'Auckland'

# correct tourney_name exchange error between Vina Del Mar and Santiago for years 2012 and 2013
df_uk.loc[(df_uk['tourney_name'] == 'Vina Del Mar') & (df_uk['Year'] == 2012), 'Location'] = 'Santiago'
df_uk.loc[(df_uk['tourney_name'] == 'Vina Del Mar') & (df_uk['Year'] == 2012), 'Tournament'] = 'Movistar Open'
df_uk.loc[(df_uk['tourney_name'] == 'Vina Del Mar') & (df_uk['Year'] == 2012), 'tourney_name'] = 'Santiago'
df_uk.loc[(df_uk['tourney_name'] == 'Vina Del Mar') & (df_uk['Year'] == 2013), 'Location'] = 'Santiago'
df_uk.loc[(df_uk['tourney_name'] == 'Vina Del Mar') & (df_uk['Year'] == 2013), 'Tournament'] = 'Movistar Open'
df_uk.loc[(df_uk['tourney_name'] == 'Vina Del Mar') & (df_uk['Year'] == 2013), 'tourney_name'] = 'Santiago'

# manually change specific Winner and Loser names to match player_list.xlsx
# change Loser name from 'Lapentti G.' to 'Lapentti N.' if df_uk['tourney_name'] == 'Buenos Aires'], df_uk['Year'] == 2009, df_uk['Winner'] == 'Nalbandian D.' and df_uk['Loser'] == 'Lapentti G.'
df_uk.loc[(df_uk['tourney_name'] == 'Buenos Aires') & (df_uk['Year'] == 2009) & (df_uk['Winner'] == 'Nalbandian D.') & (df_uk['Loser'] == 'Lapentti G.'), 'Loser'] = 'Lapentti N.'
# change Winner name from 'Schuettler R.' to 'Devvarman S.' if df_uk['tourney_name'] == 'Chennai'], df_uk['Year'] == 2009, df_uk['Winner'] == 'Schuettler R.' and df_uk['Loser'] == 'Devvarman S.'
df_uk.loc[(df_uk['tourney_name'] == 'Chennai') & (df_uk['Year'] == 2009) & (df_uk['Winner'] == 'Schuettler R.') & (df_uk['Loser'] == 'Devvarman S.'), 'Winner'] = 'Devvarman S.'
# change Loser name from 'Devvarman S.' to 'Schuettler R.' if df_uk['tourney_name'] == 'Chennai'], df_uk['Year'] == 2009, df_uk['Winner'] == 'Devvarman S.' and df_uk['Loser'] == 'Devvarman S.'
df_uk.loc[(df_uk['tourney_name'] == 'Chennai') & (df_uk['Year'] == 2009) & (df_uk['Winner'] == 'Devvarman S.') & (df_uk['Loser'] == 'Devvarman S.'), 'Loser'] = 'Schuettler R.'
# change Loser name from 'Schuettler R.' to 'Devvarman S.' if df_uk['tourney_name'] == 'Chennai'], df_uk['Year'] == 2009, df_uk['Winner'] == 'Cilic M.' and df_uk['Loser'] == 'Schuettler R.'
df_uk.loc[(df_uk['tourney_name'] == 'Chennai') & (df_uk['Year'] == 2009) & (df_uk['Winner'] == 'Cilic M.') & (df_uk['Loser'] == 'Schuettler R.'), 'Loser'] = 'Devvarman S.'
# change Winner name from 'Querrey S.' to 'Ram R.' if df_uk['tourney_name'] == 'Newport'], df_uk['Year'] == 2009, df_uk['Winner'] == 'Querrey S.' and df_uk['Loser'] == 'Ram R.'
df_uk.loc[(df_uk['tourney_name'] == 'Newport') & (df_uk['Year'] == 2009) & (df_uk['Winner'] == 'Querrey S.') & (df_uk['Loser'] == 'Ram R.'), 'Winner'] = 'Ram R.'
# change Loser name from 'Ram R.' to 'Querrey S.' if df_uk['tourney_name'] == 'Newport'], df_uk['Year'] == 2009, df_uk['Winner'] == 'Ram R.' and df_uk['Loser'] == 'Ram R.'
df_uk.loc[(df_uk['tourney_name'] == 'Newport') & (df_uk['Year'] == 2009) & (df_uk['Winner'] == 'Ram R.') & (df_uk['Loser'] == 'Ram R.'), 'Loser'] = 'Querrey S.'
# change Winner name from 'Berrettini M.' to 'Zverev A.' if df_uk['tourney_name'] == 'Tour Finals'], df_uk['Year'] == 2021, df_uk['Winner'] == 'Berrettini M.' and df_uk['Loser'] == 'Zverev A.'
df_uk.loc[(df_uk['tourney_name'] == 'Tour Finals') & (df_uk['Year'] == 2021) & (df_uk['Winner'] == 'Berrettini M.') & (df_uk['Loser'] == 'Zverev A.'), 'Winner'] = 'Zverev A.'
# change Loser name from 'Zverev A.' to 'Berrettini M.' if df_uk['tourney_name'] == 'Tour Finals'], df_uk['Year'] == 2021, df_uk['Winner'] == 'Zverev A.' and df_uk['Loser'] == 'Zverev A.'
df_uk.loc[(df_uk['tourney_name'] == 'Tour Finals') & (df_uk['Year'] == 2021) & (df_uk['Winner'] == 'Zverev A.') & (df_uk['Loser'] == 'Zverev A.'), 'Loser'] = 'Berrettini M.'
# change Loser name from 'Rochus C.' to 'Rochus O.' if df_uk['tourney_name'] == 'San Jose'], df_uk['Year'] == 2012, df_uk['Winner'] == 'Harrison R.' and df_uk['Loser'] == 'Rochus C.'
df_uk.loc[(df_uk['tourney_name'] == 'San Jose') & (df_uk['Year'] == 2012) & (df_uk['Winner'] == 'Harrison R.') & (df_uk['Loser'] == 'Rochus C.'), 'Loser'] = 'Rochus O.'
# change Loser name from 'Kuznetsov A.' to 'Kuznetsov Al.' if df_uk['tourney_name'] == 'Australian Open'], df_uk['Year'] == 2012, df_uk['Winner'] == 'Nadal R.' and df_uk['Loser'] == 'Kuznetsov A.'
df_uk.loc[(df_uk['tourney_name'] == 'Australian Open') & (df_uk['Year'] == 2012) & (df_uk['Winner'] == 'Nadal R.') & (df_uk['Loser'] == 'Kuznetsov A.'), 'Loser'] = 'Kuznetsov Al.'
# change Loser name from 'Nava E.' to 'Nava Ed.' if df_uk['Winner'] == 'Monteiro T.' and df_uk['Loser'] == 'Nava E.'
df_uk.loc[(df_uk['Winner'] == 'Monteiro T.') & (df_uk['Loser'] == 'Nava E.'), 'Loser'] = 'Nava Ed.'
# change Winner name from 'Martin A.' to 'Martin Al.' if df_uk['Year'] is between 2009 and 2010 and df_uk['Winner'] == 'Martin A.'
df_uk.loc[(df_uk['Year'] >= 2009) & (df_uk['Year'] <= 2010) & (df_uk['Winner'] == 'Martin A.'), 'Winner'] = 'Martin Al.'
# change Winner name from 'Martin A.' to 'Martin Aj.' if df_uk['Year'] is between 2016 and 2021 and df_uk['Winner'] == 'Martin A.'
df_uk.loc[(df_uk['Year'] >= 2016) & (df_uk['Year'] <= 2021) & (df_uk['Winner'] == 'Martin A.'), 'Winner'] = 'Martin Aj.'
# change Winner name from 'Martin A.' to 'Martin As.' if df_uk['Year'] >= 2022 and df_uk['Winner'] == 'Martin A.'
df_uk.loc[(df_uk['Year'] >= 2022) & (df_uk['Winner'] == 'Martin A.'), 'Winner'] = 'Martin As.'
# change Loser name from 'Martin A.' to 'Martin Al.' if df_uk['Date'] <= 2010-07-01 and df_uk['Loser'] == 'Martin A.'
df_uk.loc[(df_uk['Date'] <= '2010-07-01') & (df_uk['Loser'] == 'Martin A.'), 'Loser'] = 'Martin Al.'
# change Loser name from 'Martin A.' to 'Martin Aj.' if df_uk['Date'] between 2010-07-01 and 2022-04-01 and df_uk['Loser'] == 'Martin A.'
df_uk.loc[(df_uk['Date'] > '2010-07-01') & (df_uk['Date'] < '2022-04-01') & (df_uk['Loser'] == 'Martin A.'), 'Loser'] = 'Martin Aj.'
# change Loser name from 'Martin A.' to 'Martin As.' if df_uk['Date'] >= 2022-04-01 and df_uk['Loser'] == 'Martin A.'
df_uk.loc[(df_uk['Date'] >= '2022-04-01') & (df_uk['Loser'] == 'Martin A.'), 'Loser'] = 'Martin As.'

# replace Winner and Loser names to match player_list.xlsx
name_changes = {
    'Ali Mutawa J.M.': 'Al Mutawa J.',
    'Aragone JC': 'Aragone J.',
    'Aragone J.C.': 'Aragone J.',
    'Alawadhi O.': 'Awadhy O.',
    'Barrios M.': 'Barrios Vera M.T.',
    'Bautista R.': 'Bautista Agut R.',
    'Carreno Busta P.': 'Carreno-Busta P.',
    'De Heart R.': 'Deheart R.',
    'Del Potro J. M.': 'Del Potro J.M.',
    'Del Bonis F.': 'Delbonis F.',
    'Del Bonnis F.': 'Delbonis F.',
    'Dolgopolov O.': 'Dolgopolov A.',
    'Silva D.': 'Dutra Da Silva D.',
    'Dutra Da Silva R.': 'Dutra Silva R.',
    'Galan D.': 'Galan D.E.',
    'Granollers Pujol G.': 'Granollers G.',
    'Granollers-Pujol G.': 'Granollers G.',
    'Herbert P-H.': 'Herbert P.H.',
    'Herbert P.H': 'Herbert P.H.',
    'Hernandez-Fernandez J': 'Hernandez-Fernandez J.',
    'Kohlschreiber P..': 'Kohlschreiber P.',
    'Kuznetsov An.': 'Kuznetsov A.',
    'Lu Y.': 'Lu Y.H.',
    'Monteiro J.': 'Monteiro T.',
    'Moroni G.M.': 'Moroni G.',
    'Munoz de La Nava D.': 'Munoz-De La Nava D.',
    'Munoz De La Nava D.': 'Munoz-De La Nava D.',
    'Nava E.': 'Nava Em.',
    'Nedovyesov O.': 'Nedovyesov A.',
    "O'Connell C.": 'O Connell C.',
    'Ramos A.': 'Ramos-Vinolas A.',
    'Riba-Madrid P.': 'Riba P.',
    'Silva F.': 'Silva F.F.',
    'Ferreira Silva F.': 'Silva F.F.',
    'Stebe C-M.': 'Stebe C.M.',
    'Struff J-L.': 'Struff J.L.',
    'Tyurnev E.': 'Tiurnev E.',
    'Tseng C. H.': 'Tseng C.H.',
    'Van D. Merwe I.': 'Van Der Merwe I.',
    'Van der Merwe I.': 'Van Der Merwe I.',
    'Varillas J. P.': 'Varillas J.P.',
    'Viola Mat.': 'Viola M.',
    'Zayed M. S.': 'Zayed M.S.',
    'Zayid M. S.': 'Zayid M.S.',
    'Zayid M.': 'Zayid M.S.',
    'Zhang Ze': 'Zhang Z.',
    'Zhang Ze.': 'Zhang Z.'}
for old_name, new_name in name_changes.items():
    df_uk.loc[df_uk['Winner'] == old_name, 'Winner'] = new_name
    df_uk.loc[df_uk['Loser'] == old_name, 'Loser'] = new_name

# insert new columns 'winner_id' and 'loser_id' based on 'uk' and 'id' columns of player_list.xlsx 
df_player = pd.read_excel(folder_path + '/_players_list.xlsx')
players = df_player.set_index(['uk'])['id'].to_dict()
def get_player_id(player_name):
    player_id = players.get(player_name)
    return player_id if player_id else 'NA'
df_uk.insert(df_uk.columns.get_loc('Winner'), 'winner_id', df_uk['Winner'].apply(get_player_id))
df_uk.insert(df_uk.columns.get_loc('Loser'), 'loser_id', df_uk['Loser'].apply(get_player_id))

# manually correct the round R16 to QF for the match between Molcan A. and Coria F.
df_uk.loc[(df_uk['tourney_name'] == 'Lyon') & (df_uk['Year'] == 2022) & (df_uk['round'] == 'R16') & (df_uk['Winner'] == 'Molcan A.') & (df_uk['Loser'] == 'Coria F.'), ['Round', 'round']] = ['Quarterfinals', 'QF']
# manually correct the round QF to R16 for the match between Guinard M. and Mmoh M.
df_uk.loc[(df_uk['tourney_name'] == 'Lyon') & (df_uk['Year'] == 2022) & (df_uk['round'] == 'QF') & (df_uk['Winner'] == 'Guinard M.') & (df_uk['Loser'] == 'Mmoh M.'), ['Round', 'round']] = ['2nd Round', 'R16']

# manually correct the Comment field for the match between Istomin D. and Baghdatis M.
df_uk.loc[(df_uk['tourney_name'] == 'Nottingham') & (df_uk['Year'] == 2015) & (df_uk['Winner'] == 'Istomin D.') & (df_uk['Loser'] == 'Baghdatis M.'), 'Comment'] = 'Retired'

# replace tourney_name 'Tour Finals' with 'Tour Finals London' (2009 to 2020) and 'Tour Finals Turin' (2021 and 2022)
df_uk.loc[(df_uk['tourney_name'] == 'Tour Finals') & (~df_uk['Year'].isin([2021, 2022])), 'tourney_name'] = 'Tour Finals London'
df_uk.loc[(df_uk['tourney_name'] == 'Tour Finals') & (df_uk['Year'].isin([2021, 2022])), 'tourney_name'] = 'Tour Finals Turin'

# change Comment field from 'Sched' to 'Completed'
df_uk.loc[df_uk['Comment'] == 'Sched', 'Comment'] = 'Completed'
#change Comment field from 'Walkover' to 'Not Played'
df_uk.loc[df_uk['Comment'] == 'Walkover', 'Comment'] = 'Not Played'
# change Comment field from 'Rrtired' or 'Retired' or 'Disqualified' or 'Awarded' to 'Unfinished' 
df_uk.loc[df_uk['Comment'].isin(['Rrtired', 'Retired', 'Disqualified', 'Awarded']), 'Comment'] = 'Unfinished'

# output df into an xlsx file in "data" folder
df_uk.to_excel(folder_path + '/tennis_data_all.xlsx', index=False)

df_uk.head()

(59764, 55)
(35447, 55)


,ATP,Location,Tournament,tourney_name,Date,Year,Series,Court,Surface,Round,round,Best of,winner_id,Winner,loser_id,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,UBW,UBL,CBW,CBL,IWW,IWL,SBW,SBL,B&WW,B&WL,GBW,GBL
9302,1,Brisbane,Brisbane International,Brisbane,2009-01-04,2009,ATP250,Outdoor,Hard,1st Round,R32,3.00,104755,Gasquet R.,102967,Gicquel M.,23.00,50.00,2320.00,1428.00,4.00,6.00,6.00,2.00,6.00,2.00,NaN,NaN,NaN,NaN,2.00,1.00,Completed,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,NaN,NaN,NaN,NaN,1.27,3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9303,1,Brisbane,Brisbane International,Brisbane,2009-01-04,2009,ATP250,Outdoor,Hard,1st Round,R32,3.00,103758,Dent T.,104327,Darcis S.,865.00,59.00,28.00,1362.00,7.00,6.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,Completed,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9304,1,Brisbane,Brisbane International,Brisbane,2009-01-04,2009,ATP250,Outdoor,Hard,1st Round,R32,3.00,103898,Benneteau J.,104068,Ginepri R.,40.00,49.00,1608.00,1480.00,3.00,6.00,6.00,3.00,7.00,5.00,NaN,NaN,NaN,NaN,2.00,1.00,Completed,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,NaN,NaN,NaN,NaN,1.96,1.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9305,1,Brisbane,Brisbane International,Brisbane,2009-01-04,2009,ATP250,Outdoor,Hard,1st Round,R32,3.00,104417,Soderling R.,105023,Querrey S.,17.00,36.00,2650.00,1740.00,6.00,3.00,6.00,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,Completed,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,NaN,NaN,NaN,NaN,1.40,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9306,1,Brisbane,Brisbane International,Brisbane,2009-01-05,2009,ATP250,Outdoor,Hard,1st Round,R32,3.00,103813,Nieminen J.,104571,Baghdatis M.,41.00,96.00,1600.00,950.00,5.00,7.00,6.00,1.00,6.00,4.00,NaN,NaN,NaN,NaN,2.00,1.00,Completed,2.00,1.72,1.95,1.85,1.83,1.83,NaN,NaN,2.00,1.80,NaN,NaN,NaN,NaN,1.88,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Jeff Sackmann

In [4]:
# Set the directory path where the CSV files are located
folder_path = "_data"

# Get a list of all CSV files in the directory
file_list = [file for file in os.listdir(folder_path) if file.startswith("atp_matches_20") and file.endswith(".csv")]

# Initialize an empty list to store the dataframes
dataframes = []
single_df = []
df_jeff = []

# Iterate over each CSV file and read it into a dataframe
for file in file_list:
    file_path = os.path.join(folder_path, file)
    year_pos = file.find("20")
    year = file[year_pos:year_pos+4]
    single_df = pd.read_csv(file_path)
    date_index = single_df.columns.get_loc('tourney_date')
    single_df.insert(date_index + 1, 'Year', int(year))
    dataframes.append(single_df)

# Merge all dataframes into a single dataframe
df_jeff = pd.concat(dataframes, ignore_index=True)

# Print the merged dataframe
print(df_jeff.head())

  tourney_id tourney_name surface  draw_size tourney_level  tourney_date  Year  match_num  winner_id winner_seed winner_entry     winner_name     winner_hand  winner_ht winner_ioc  winner_age  \
0  2019-M020   Brisbane     Hard      32            A         20181231    2019     300      105453       2.00         NaN           Kei Nishikori       R       178.00       JPN       29.00      
1  2019-M020   Brisbane     Hard      32            A         20181231    2019     299      106421       4.00         NaN         Daniil Medvedev       R       198.00       RUS       22.80      
2  2019-M020   Brisbane     Hard      32            A         20181231    2019     298      105453       2.00         NaN           Kei Nishikori       R       178.00       JPN       29.00      
3  2019-M020   Brisbane     Hard      32            A         20181231    2019     297      104542        NaN          PR      Jo-Wilfried Tsonga       R       188.00       FRA       33.70      
4  2019-M020   Brisbane  

In [5]:
print(df_jeff.shape)

# trim tourney_name field to remove leading and trailing spaces
df_jeff['tourney_name'] = df_jeff['tourney_name'].str.strip()
df_jeff['winner_name'] = df_jeff['winner_name'].str.strip()
df_jeff['loser_name'] = df_jeff['loser_name'].str.strip()

# remove data points with Date <= 2008 and Date > 2022
df_jeff['tourney_date'] = pd.to_datetime(df_jeff['tourney_date'], format='%Y%m%d')
df_jeff['tourney_date'] = df_jeff['tourney_date'].dt.strftime('%Y-%m-%d')
#df_jeff = df_jeff[(df_jeff['tourney_date'] > '2008-12-31') & (df_jeff['tourney_date'] <= '2022-12-31')]
df_jeff = df_jeff[(df_jeff['Year'] > 2008) & (df_jeff['Year'] <= 2022)]
df_jeff = df_jeff.sort_values(by=['tourney_date', 'tourney_id', 'match_num'])
print(df_jeff.shape)

# remove data points with tourney_name = 'Dusseldorf' and Year = 2009 or 2010 or 2011 or 2012 #World Team Cup
df_jeff = df_jeff[~((df_jeff['tourney_name'] == 'Dusseldorf') & (df_jeff['Year'] >= 2009) & (df_jeff['Year'] <= 2012))]

# remove data points with tourney_level = 'D' #Davis Cup
df_jeff = df_jeff[df_jeff['tourney_level'] != 'D']
print(df_jeff.shape)

# remove data points with tourney_name = 'Atp Cup'
df_jeff = df_jeff[df_jeff['tourney_name'] != 'Atp Cup']
print(df_jeff.shape)

# remove data points with tourney_name = *Olympics*
df_jeff = df_jeff[~df_jeff['tourney_name'].str.contains('Olympics')]
print(df_jeff.shape)

# remove data points with tourney_name = 'Laver Cup'
df_jeff = df_jeff[df_jeff['tourney_name'] != 'Laver Cup']
print(df_jeff.shape)

# remove data points with tourney_name = 'NextGen Finals'
df_jeff = df_jeff[df_jeff['tourney_name'] != 'NextGen Finals']
print(df_jeff.shape)

# replace tourney_name values to match df_uk
values_to_replace = {
    'Nur-Sultan': 'Astana',
    'ATP Rio de Janeiro': 'Rio de Janeiro',
    'St Petersburg': 'St. Petersburg',
    'Us Open': 'US Open'}
df_jeff['tourney_name'] = df_jeff['tourney_name'].replace(values_to_replace)

# replace tourney_name 'Tour Finals' with 'Tour Finals London' (2009 to 2020) and 'Tour Finals Turin' (2021 and 2022)
df_jeff.loc[(df_jeff['tourney_name'] == 'Tour Finals') & (~df_jeff['Year'].isin([2021, 2022])), 'tourney_name'] = 'Tour Finals London'
df_jeff.loc[(df_jeff['tourney_name'] == 'Tour Finals') & (df_jeff['Year'].isin([2021, 2022])), 'tourney_name'] = 'Tour Finals Turin'

# add match_id column as the unique identifier for each match
df_jeff['match_num_str'] = df_jeff['match_num'].apply(lambda x: str(x).zfill(4))
df_jeff['match_id'] = df_jeff['tourney_id'] + '-' + df_jeff['match_num_str']
df_jeff = df_jeff.drop('match_num_str', axis=1)
col = df_jeff.pop('match_id')
df_jeff.insert(0, col.name, col)

# output df into an xlsx file in "data" folder
df_jeff.to_excel(folder_path + '/atp_matches_all.xlsx', index=False)

df_jeff.head()

(70419, 50)
(39594, 50)
(35926, 50)
(35756, 50)
(35564, 50)
(35524, 50)
(35447, 50)


,match_id,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,Year,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
15324,2009-339-0001,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,1,105208,NaN,NaN,Ernests Gulbis,R,190.00,LAT,20.30,104925,1.00,NaN,Novak Djokovic,R,188.00,SRB,21.60,6-4 6-4,3,R32,90.00,4.00,2.00,61.00,38.00,28.00,11.00,10.00,2.00,4.00,3.00,4.00,65.00,31.00,21.00,14.00,10.00,9.00,13.00,53.00,1408.00,3.00,10590.00
15325,2009-339-0002,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,2,103908,NaN,NaN,Paul Henri Mathieu,R,185.00,FRA,26.90,104559,NaN,Q,Teymuraz Gabashvili,R,188.00,RUS,23.60,6-3 6-3,3,R32,73.00,9.00,2.00,48.00,28.00,26.00,9.00,9.00,2.00,3.00,3.00,4.00,50.00,25.00,17.00,10.00,9.00,5.00,9.00,31.00,2000.00,65.00,1226.00
15326,2009-339-0003,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,3,105453,NaN,NaN,Kei Nishikori,R,178.00,JPN,19.00,104035,NaN,Q,Bobby Reynolds,R,183.00,USA,26.40,6-3 6-2,3,R32,59.00,5.00,4.00,45.00,23.00,19.00,14.00,9.00,0.00,1.00,5.00,1.00,53.00,29.00,18.00,8.00,8.00,4.00,8.00,61.00,1280.00,68.00,1144.00
15327,2009-339-0004,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,4,104607,5.00,NaN,Tomas Berdych,R,196.00,CZE,23.30,105454,NaN,WC,Brydan Klein,R,183.00,AUS,19.00,6-0 6-4,3,R32,67.00,5.00,2.00,50.00,33.00,28.00,6.00,8.00,4.00,5.00,0.00,2.00,51.00,27.00,15.00,7.00,8.00,3.00,8.00,20.00,2430.00,248.00,304.00
15328,2009-339-0005,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,5,104269,3.00,NaN,Fernando Verdasco,L,188.00,ESP,25.10,106071,NaN,WC,Bernard Tomic,R,193.00,AUS,16.20,6-4 6-2,3,R32,65.00,1.00,2.00,50.00,39.00,31.00,5.00,9.00,3.00,4.00,3.00,1.00,56.00,34.00,19.00,9.00,9.00,9.00,13.00,15.00,2830.00,764.00,42.00


### Tennis-Data vs. Jeff Sackmann

In [6]:
# Group df_uk by 'tourney_name' and count number of rows
df_uk_grouped = df_uk.groupby('tourney_name').size().reset_index(name='count_uk')

# Group df_jeff by 'tourney_name' and count number of rows
df_jeff_grouped = df_jeff.groupby('tourney_name').size().reset_index(name='count_jeff')

# Merge the two groupings
merged = pd.merge(df_uk_grouped, df_jeff_grouped, on='tourney_name', how='outer')

# Find rows where counts are different
differences = merged[merged['count_uk'] != merged['count_jeff']]
print(differences)

Empty DataFrame
Columns: [tourney_name, count_uk, count_jeff]
Index: []


In [7]:
# Group by 'tourney_name' and 'Year' and count number of rows
df_uk_grouped = df_uk.groupby(['tourney_name', 'Year']).size().reset_index(name='count_uk')

# Group by 'tourney_name' and 'Year' and count number of rows
df_jeff_grouped = df_jeff.groupby(['tourney_name', 'Year']).size().reset_index(name='count_jeff')

# Merge the two groupings
merged = pd.merge(df_uk_grouped, df_jeff_grouped, left_on=['tourney_name', 'Year'], right_on=['tourney_name', 'Year'], how='outer')

# Find rows where counts are different
differences = merged[merged['count_uk'] != merged['count_jeff']]
print(differences)

Empty DataFrame
Columns: [tourney_name, Year, count_uk, count_jeff]
Index: []


In [8]:
# Group by 'tourney_name', 'Year' and 'round' and count number of rows
df_uk_grouped = df_uk.groupby(['tourney_name', 'Year', 'round']).size().reset_index(name='count_uk')

# Group by 'tourney_name', 'Year' and 'round' and count number of rows
df_jeff_grouped = df_jeff.groupby(['tourney_name', 'Year', 'round']).size().reset_index(name='count_jeff')

# Merge the two groupings
merged = pd.merge(df_uk_grouped, df_jeff_grouped, left_on=['tourney_name', 'Year', 'round'], right_on=['tourney_name', 'Year', 'round'], how='outer')

# Find rows where counts are different
differences = merged[merged['count_uk'] != merged['count_jeff']]
print(differences)

Empty DataFrame
Columns: [tourney_name, Year, round, count_uk, count_jeff]
Index: []


In [9]:
# Group by 'tourney_name', 'Year', 'round' and 'winner_id' count number of rows
df_uk_grouped = df_uk.groupby(['tourney_name', 'Year', 'round', 'winner_id', 'loser_id']).size().reset_index(name='count_uk')

# Group by 'tourney_name', 'Year' and 'round' and count number of rows
df_jeff_grouped = df_jeff.groupby(['tourney_name', 'Year', 'round', 'winner_id', 'loser_id']).size().reset_index(name='count_jeff')

# Merge the two groupings
merged = pd.merge(df_uk_grouped, df_jeff_grouped, left_on=['tourney_name', 'Year', 'round', 'winner_id', 'loser_id'], right_on=['tourney_name', 'Year', 'round', 'winner_id', 'loser_id'], how='outer')

# Find rows where counts are NaN or different
nans = merged[merged['count_uk'].isna() | merged['count_jeff'].isna()]
differences = merged[merged['count_uk'] != merged['count_jeff']]
print(nans)
print(differences)

Empty DataFrame
Columns: [tourney_name, Year, round, winner_id, loser_id, count_uk, count_jeff]
Index: []
Empty DataFrame
Columns: [tourney_name, Year, round, winner_id, loser_id, count_uk, count_jeff]
Index: []


### Merged Dataframe

In [10]:
# Merge df_uk and df_jeff on the fields ['tourney_name', 'Year', 'round', 'winner_id']
df = pd.merge(df_jeff, df_uk, left_on=['tourney_name', 'Year', 'round', 'winner_id', 'loser_id'], right_on=['tourney_name', 'Year', 'round', 'winner_id', 'loser_id'], how='inner')
print(df.shape)

# Correct dates of some rows by adding one year to the 'Date' column where 'tourney_name' is 'Beijing' or 'Tokyo' and 'Year' is 2013
df['Date'] = pd.to_datetime(df['Date'])
df.loc[(df['tourney_name'].isin(['Beijing', 'Tokyo'])) & (df['Year'] == 2013), 'Date'] = df.loc[(df['tourney_name'].isin(['Beijing', 'Tokyo'])) & (df['Year'] == 2013), 'Date'] + pd.DateOffset(years=1)

# Define 'round_order' column based on the 'round' column
round_order = {'R128': 1, 'R64': 2, 'R32': 3, 'R16': 4, 'QF': 5, 'SF': 6, 'F': 7}
df['round_order'] = df['round'].map(round_order)

# Sort the DataFrame by 'Date' and the mapped order numbers for the 'round' column
df = df.sort_values(by=['Date', 'round_order'], ascending=True)

# Drop the temporary column used for sorting
df.drop('round_order', axis=1, inplace=True)

df.head()

(35447, 105)


,match_id,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,Year,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,UBW,UBL,CBW,CBL,IWW,IWL,SBW,SBL,B&WW,B&WL,GBW,GBL
10,2009-339-0011,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,11,103898,NaN,NaN,Julien Benneteau,R,185.00,FRA,27.00,104068,NaN,NaN,Robby Ginepri,R,183.00,USA,26.20,3-6 6-3 7-5,3,R32,127.00,10.00,9.00,99.00,62.00,41.00,17.00,15.00,2.00,6.00,8.00,2.00,88.00,53.00,32.00,21.00,15.00,4.00,9.00,40.00,1608.00,49.00,1480.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Benneteau J.,Ginepri R.,40.00,49.00,1608.00,1480.00,3.00,6.00,6.00,3.00,7.00,5.00,NaN,NaN,NaN,NaN,2.00,1.00,Completed,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,NaN,NaN,NaN,NaN,1.96,1.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2009-339-0012,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,12,104417,4.00,NaN,Robin Soderling,R,193.00,SWE,24.30,105023,NaN,NaN,Sam Querrey,R,198.00,USA,21.20,6-3 6-3,3,R32,55.00,7.00,2.00,52.00,30.00,26.00,13.00,9.00,0.00,0.00,4.00,3.00,49.00,27.00,20.00,9.00,9.00,2.00,5.00,17.00,2650.00,36.00,1740.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Soderling R.,Querrey S.,17.00,36.00,2650.00,1740.00,6.00,3.00,6.00,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,Completed,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,NaN,NaN,NaN,NaN,1.40,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2009-339-0013,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,13,104755,7.00,NaN,Richard Gasquet,R,185.00,FRA,22.50,102967,NaN,NaN,Marc Gicquel,R,188.00,FRA,31.70,4-6 6-2 6-2,3,R32,133.00,6.00,5.00,91.00,42.00,29.00,26.00,13.00,3.00,5.00,7.00,6.00,100.00,45.00,30.00,23.00,13.00,13.00,18.00,23.00,2320.00,50.00,1428.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Gasquet R.,Gicquel M.,23.00,50.00,2320.00,1428.00,4.00,6.00,6.00,2.00,6.00,2.00,NaN,NaN,NaN,NaN,2.00,1.00,Completed,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,NaN,NaN,NaN,NaN,1.27,3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2009-339-0014,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,14,103758,NaN,NaN,Taylor Dent,R,188.00,USA,27.70,104327,NaN,NaN,Steve Darcis,R,178.00,BEL,24.80,7-6(0) 6-2,3,R32,86.00,8.00,5.00,68.00,37.00,31.00,13.00,10.00,6.00,8.00,4.00,2.00,70.00,46.00,24.00,14.00,10.00,1.00,5.00,865.00,28.00,59.00,1362.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Dent T.,Darcis S.,865.00,59.00,28.00,1362.00,7.00,6.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,Completed,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-339-0005,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,5,104269,3.00,NaN,Fernando Verdasco,L,188.00,ESP,25.10,106071,NaN,WC,Bernard Tomic,R,193.00,AUS,16.20,6-4 6-2,3,R32,65.00,1.00,2.00,50.00,39.00,31.00,5.00,9.00,3.00,4.00,3.00,1.00,56.00,34.00,19.00,9.00,9.00,9.00,13.00,15.00,2830.00,764.00,42.00,1,Brisbane,Brisbane International,2009-01-05,ATP250,Outdoor,Hard,1st Round,3.00,Verdasco F.,Tomic B.,15.00,764.00,2830.00,42.00,6.00,4.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,Completed,1.06,8.00,1.06,9.52,1.04,9.00,NaN,NaN,1.03,11.00,NaN,NaN,NaN,NaN,1.04,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Export df into an xlsx file in "data" folder
df.to_excel(folder_path + '/_matches_list_v0.xlsx', index=False)